# SunStone 3D FDTD Demo — Young's Double Slit

This notebook runs a 3D double-slit simulation using Meep and compares the measured screen intensity to a 2D Fraunhofer theory curve along the mid-plane ($z=0$).

Note: 3D runs are heavier. Keep resolution modest for quick iteration.

## Prerequisites
- SunStone API running locally (default: http://127.0.0.1:8000).
- `meep` available to the worker environment.
- `numpy`, `matplotlib`, `requests` installed.

In [ ]:
import json
import time
import csv
from io import StringIO
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
import requests

In [ ]:
API_BASE = "http://127.0.0.1:8000"

def create_project(name: str):
    resp = requests.post(f"{API_BASE}/projects", json={"name": name})
    resp.raise_for_status()
    return resp.json()

def create_run(project_id: str, spec: dict):
    resp = requests.post(f"{API_BASE}/projects/{project_id}/runs", json={"spec": spec})
    resp.raise_for_status()
    return resp.json()

def submit_run(run_id: str, backend: str = "meep", python_executable: str | None = None):
    payload = {"mode": "local", "backend": backend}
    if python_executable:
        payload["python_executable"] = python_executable
    resp = requests.post(f"{API_BASE}/runs/{run_id}/submit", json=payload)
    resp.raise_for_status()
    return resp.json()

def get_run(run_id: str):
    resp = requests.get(f"{API_BASE}/runs/{run_id}")
    resp.raise_for_status()
    return resp.json()

def list_artifacts(run_id: str):
    resp = requests.get(f"{API_BASE}/runs/{run_id}/artifacts")
    resp.raise_for_status()
    return resp.json()["artifacts"]

def download_artifact(run_id: str, path: str):
    resp = requests.get(f"{API_BASE}/runs/{run_id}/artifacts/{path}")
    resp.raise_for_status()
    return resp.text

In [ ]:
# 3D double-slit parameters (meters)
cell_x = 20e-6
cell_y = 12e-6
cell_z = 6e-6
pml = 2e-6
resolution = 24

slit_width = 1.0e-6
slit_sep = 3.0e-6
barrier_thickness = 0.5e-6
barrier_depth = 2.0e-6

# Frequency (Hz) ~ 1 micron wavelength in vacuum
center_freq = 3.0e14
fwidth = 0.02 * center_freq

half_h = cell_y / 2
top_height = half_h - (slit_sep / 2 + slit_width / 2)
mid_height = slit_sep - slit_width
bottom_height = top_height

top_center = (slit_sep / 2 + slit_width / 2 + top_height / 2)
mid_center = 0.0
bottom_center = -(slit_sep / 2 + slit_width / 2 + bottom_height / 2)

pec_blocks = [
code
python
project = create_project("double-slit-3d")
run = create_run(project["id"], spec)
submit_run(run["id"], backend="meep")

# Poll run status
for _ in range(90):
    status = get_run(run["id"])
    print(status["status"])
    if status["status"] in ("succeeded", "failed"):
        break
    time.sleep(3)
code
python
artifacts = list_artifacts(run["id"])
monitor_paths = [a["path"] for a in artifacts if a["path"].startswith("outputs/monitors/")]
len(monitor_paths)
code
python
def load_monitor_values(path: str):
    txt = download_artifact(run["id"], path)
    rows = list(csv.DictReader(StringIO(txt)))
    if not rows:
        return None
    # Use last 20% of samples to compute RMS-like intensity
    tail = rows[int(len(rows) * 0.8):]
    vals = [float(r.get("Ez", 0.0)) for r in tail]
    return float(np.mean(np.square(vals)))

monitor_paths_sorted = sorted(monitor_paths)
intensity = np.array([load_monitor_values(p) for p in monitor_paths_sorted])
intensity = intensity / np.max(intensity)
: 
,
: {
: 

: [
,
,
300000000
,
,
,
0.45
,
,
,
,
2
2
,
,
,
4
,
1000000
Simulation (3D, z=0)")
plt.plot(y * 1e6, I_theory, label="Theory", linestyle="--")
plt.xlabel("Screen Y (µm)")
plt.ylabel("Normalized intensity")
plt.title("Double-slit interference pattern (3D mid-plane)")
plt.legend()
plt.tight_layout()
plt.show()